In [266]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [267]:
column_names = list(range(0,4))
column_names.append('Classification')

df = pd.read_csv("iris.data", names = column_names) 
df

,0,1,2,3,Classification
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [268]:
df.dtypes

0                 float64
1                 float64
2                 float64
3                 float64
Classification     object
dtype: object

In [269]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
Classification    0
dtype: int64

In [270]:
df.nunique()

0                 35
1                 23
2                 43
3                 22
Classification     3
dtype: int64

In [271]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['Classification'] = label_encoder.fit_transform(df['Classification'])
df

,0,1,2,3,Classification
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [272]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [273]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (4,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0

for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
5/5 [==============================] - 1s 119ms/step - loss: 1.0083 - accuracy: 0.5259 - precision: 0.5745 - recall: 0.2000 - val_loss: 0.9392 - val_accuracy: 0.6000 - val_precision: 0.6364 - val_recall: 0.4667
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 0.8146 - accuracy: 0.6741 - precision: 0.8269 - recall: 0.6370 - val_loss: 0.6797 - val_accuracy: 0.8667 - val_precision: 1.0000 - val_recall: 0.4667
Epoch 3/50
5/5 [==============================] - 0s 11ms/step - loss: 0.6786 - accuracy: 0.7333 - precision: 0.7250 - recall: 0.4296 - val_loss: 0.5572 - val_accuracy: 0.8667 - val_precision: 1.0000 - val_recall: 0.6000
Epoch 4/50
5/5 [==============================] - 0s 10ms/step - loss: 0.5772 - accuracy: 0.7333 - precision: 0.8265 - recall: 0.6000 - val_loss: 0.5586 - val_accuracy: 0.6000 - val_precision: 0.6923 - val_recall: 0.6000
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - loss: 0.5005 - accuracy: 0.6815 - precision: 0.7913

Epoch 38/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0910 - accuracy: 0.9704 - precision: 0.9704 - recall: 0.9704 - val_loss: 0.1333 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 39/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0891 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.1287 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 40/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0896 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.1249 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 41/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0872 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.1258 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 42/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0852 - accuracy: 0.9852 - precision: 0.

1/1 [==============================] - 0s 17ms/step
Epoch 1/50
5/5 [==============================] - 0s 19ms/step - loss: 0.0782 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.0030 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0838 - accuracy: 0.9704 - precision: 0.9704 - recall: 0.9704 - val_loss: 0.0028 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0765 - accuracy: 0.9704 - precision: 0.9704 - recall: 0.9704 - val_loss: 0.0031 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0747 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.0025 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - lo

Epoch 15/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0955 - accuracy: 0.9704 - precision: 0.9704 - recall: 0.9704 - val_loss: 0.0518 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 16/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0644 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.1055 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 17/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0713 - accuracy: 0.9630 - precision: 0.9630 - recall: 0.9630 - val_loss: 0.0479 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 18/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0619 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0476 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 19/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0651 - accuracy: 0.9778 - precision: 0.9778

Epoch 12/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0578 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.1510 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 13/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0545 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.0707 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 14/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0538 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.0894 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 15/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0520 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.1133 - val_accuracy: 0.9333 - val_precision: 0.9333 - val_recall: 0.9333
Epoch 16/50
1/1 [==============================] - 0s 108ms/step
Epoch 1/50
5/5 [==============================] - 0

5/5 [==============================] - 0s 10ms/step - loss: 0.0651 - accuracy: 0.9630 - precision: 0.9630 - recall: 0.9630 - val_loss: 0.0479 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0584 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0214 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0586 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0164 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 6/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0665 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0236 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 7/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0803 - accuracy: 0.9556 - precision: 0.9556 - recall: 0.95

Epoch 16/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0605 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0049 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 17/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0644 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0033 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 18/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0588 - accuracy: 0.9778 - precision: 0.9778 - recall: 0.9778 - val_loss: 0.0069 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 19/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0584 - accuracy: 0.9852 - precision: 0.9852 - recall: 0.9852 - val_loss: 0.0046 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 20/50
5/5 [==============================] - 0s 9ms/step - loss: 0.0549 - accuracy: 0.9852 - precision: 0.9

In [274]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  0.9600000000000002
Average Precision Score:  0.9600000000000002
Average Recall Score:  0.9600000000000002
Average F1-Score:  0.9600000000000002
